In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-1.4b-deduped")

def count_hf_tokens(text):
    tokens = tokenizer.tokenize(text)
    return len(tokens)

In [1]:
import random
import itertools
import json

variables = ['A', 'B', 'C', 'D']

def random_negation():
    return random.choice([True, False])

def premise_to_str(antecedent, consequent, ant_neg, cons_neg):
    ant_str = ('NOT ' if ant_neg else '') + antecedent
    cons_str = ('NOT ' if cons_neg else '') + consequent
    return f"If {ant_str} then {cons_str}"

def evaluate_premise(antecedent_val, consequent_val, ant_neg, cons_neg):
    # evaluate "If ant then cons" logical implication
    ant = not antecedent_val if ant_neg else antecedent_val
    cons = not consequent_val if cons_neg else consequent_val
    return (not ant) or cons  # standard material implication

def evaluate_query(premises, facts, query_var, query_neg):
    # We'll do a simple forward inference for this small example:
    # For more complex logic, a proper logic solver would be needed.
    # Here: check if query_var can be inferred true or false logically
    # For demo: check if query matches facts directly or contradicts premises

    # For simplicity, we just check query against facts (no deep inference)
    val = facts[query_var]
    if query_neg:
        val = not val
    return val

def generate_instance():
    # Pick 3 unique premises
    premises = []
    used_pairs = set()
    while len(premises) < 3:
        ant = random.choice(variables)
        cons = random.choice(variables)
        if ant == cons:
            continue
        if (ant, cons) in used_pairs:
            continue
        used_pairs.add((ant, cons))
        ant_neg = random_negation()
        cons_neg = random_negation()
        premises.append( (ant, cons, ant_neg, cons_neg) )

    # Generate facts for variables
    facts = {v: random.choice([True, False]) for v in variables}

    # Pick query variable and negation
    query_var = random.choice(variables)
    query_neg = random_negation()

    # Evaluate label: check if query holds given facts (simplified)
    label = evaluate_query(premises, facts, query_var, query_neg)

    # Prepare strings
    premises_str = [premise_to_str(*p) for p in premises]
    facts_str = ', '.join([f"{k}={'True' if v else 'False'}" for k,v in facts.items()])
    query_str = f"Is {('NOT ' if query_neg else '')}{query_var} true?"

    clean = f"Premises: {'; '.join(premises_str)}. Facts: {facts_str}. Query: {query_str}"
    
    # Create corrupted version: flip negation of consequent in one random premise
    corrupted_premises = premises.copy()
    corrupted_idx = random.randint(0, 2)
    ant, cons, ant_neg, cons_neg = corrupted_premises[corrupted_idx]
    corrupted_premises[corrupted_idx] = (ant, cons, ant_neg, not cons_neg)  # flip consequent negation
    corrupted_premises_str = [premise_to_str(*p) for p in corrupted_premises]
    corrupted = f"Premises: {'; '.join(corrupted_premises_str)}. Facts: {facts_str}. Query: {query_str}"

    return {
        "clean": clean,
        "corrupted": corrupted,
        "label": label
    }

def generate_dataset(n):
    dataset = []
    for _ in range(n):
        instance = generate_instance()
        dataset.append(instance)
    return dataset

if __name__ == "__main__":
    n_instances = 6000
    data = generate_dataset(n_instances)

    # Save to JSONL file
    with open("logic_multistep_deduction.jsonl", "w") as f:
        for entry in data:
            f.write(json.dumps(entry) + "\n")

    print(f"Generated {n_instances} multi-step logic deduction instances.")


Generated 6000 multi-step logic deduction instances.
